# DESS Model - Kaggle P100 GPU Setup
## Aspect Sentiment Triplet Extraction with Contrastive Learning

This notebook runs the DESS (D2E2S) model with **Contrastive Learning** for improved entity-opinion pairing.

**Latest Update**: Added contrastive learning module for +0.5-0.8% Triplet F1 improvement

## 1. Clone Repository

In [ ]:
!git clone https://github.com/VishalRepos/DESS-improved.git
%cd DESS-improved/Codebase
!echo "\n=== Latest Commits ==="
!git log --oneline -5

## 2. Check GPU Availability

In [ ]:
!nvidia-smi

## 3. Install Dependencies

In [ ]:
# Install compatible versions
!pip install -q numpy==1.26.4
!pip install -q transformers==4.36.0
!pip install -q torch==2.1.0
!pip install -q Jinja2==3.1.2
!pip install -q tensorboardX==2.6
!pip install -q tqdm==4.65.0
!pip install -q scikit-learn==1.3.2
!pip install -q 'spacy>=3.7.2,<3.8.0'
!pip install -q matplotlib==3.8.0
!pip install -q torch_geometric==2.4.0
!pip install -q 'pydantic>=2.7.0'

# Restart kernel to apply changes
import os
os.kill(os.getpid(), 9)

## 4. Import Libraries and Check Setup

In [ ]:
import torch
import os

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 5. Verify Data Files

In [ ]:
!ls -la data/14res/
!ls -la data/14lap/
!ls -la data/15res/
!ls -la data/16res/

## 6. Quick Test - Contrastive Learning (1 epoch)
### Test new contrastive learning module with Enhanced Semantic GCN

In [ ]:
!python train.py \
    --seed 42 \
    --max_span_size 8 \
    --batch_size 16 \
    --epochs 1 \
    --dataset 14res \
    --pretrained_deberta_name microsoft/deberta-v3-base \
    --deberta_feature_dim 768 \
    --hidden_dim 384 \
    --emb_dim 768 \
    --use_enhanced_semgcn \
    --use_contrastive \
    --contrastive_weight 0.1

## 7. Full Training - Enhanced SemGCN + Contrastive Learning (120 epochs)
### Expected: Triplet F1 77.6-78.0% (+0.5-0.8% improvement)

In [ ]:
!python train.py \
    --seed 42 \
    --max_span_size 8 \
    --batch_size 16 \
    --epochs 120 \
    --dataset 14res \
    --pretrained_deberta_name microsoft/deberta-v3-base \
    --deberta_feature_dim 768 \
    --hidden_dim 384 \
    --emb_dim 768 \
    --use_enhanced_semgcn \
    --use_contrastive \
    --contrastive_weight 0.1

## 8. Baseline Comparison - Enhanced SemGCN Only (No Contrastive)
### For comparison: Expected Triplet F1 77.14%

In [ ]:
# Uncomment to run baseline for comparison
# !python train.py \
#     --seed 42 \
#     --max_span_size 8 \
#     --batch_size 16 \
#     --epochs 120 \
#     --dataset 14res \
#     --pretrained_deberta_name microsoft/deberta-v3-base \
#     --deberta_feature_dim 768 \
#     --hidden_dim 384 \
#     --emb_dim 768 \
#     --use_enhanced_semgcn

## 9. Hyperparameter Tuning - Different Contrastive Weights

### 9a. Lower Weight (0.05) - Less Influence

In [ ]:
# Uncomment to try lower contrastive weight
# !python train.py \
#     --seed 42 \
#     --max_span_size 8 \
#     --batch_size 16 \
#     --epochs 120 \
#     --dataset 14res \
#     --pretrained_deberta_name microsoft/deberta-v3-base \
#     --deberta_feature_dim 768 \
#     --hidden_dim 384 \
#     --emb_dim 768 \
#     --use_enhanced_semgcn \
#     --use_contrastive \
#     --contrastive_weight 0.05

### 9b. Higher Weight (0.2) - More Influence

In [ ]:
# Uncomment to try higher contrastive weight
# !python train.py \
#     --seed 42 \
#     --max_span_size 8 \
#     --batch_size 16 \
#     --epochs 120 \
#     --dataset 14res \
#     --pretrained_deberta_name microsoft/deberta-v3-base \
#     --deberta_feature_dim 768 \
#     --hidden_dim 384 \
#     --emb_dim 768 \
#     --use_enhanced_semgcn \
#     --use_contrastive \
#     --contrastive_weight 0.2

## 10. Test on Other Datasets

### 10a. Laptop 2014 Dataset

In [ ]:
# Uncomment to test on Laptop 2014 dataset
# !python train.py \
#     --seed 42 \
#     --max_span_size 8 \
#     --batch_size 16 \
#     --epochs 120 \
#     --dataset 14lap \
#     --pretrained_deberta_name microsoft/deberta-v3-base \
#     --deberta_feature_dim 768 \
#     --hidden_dim 384 \
#     --emb_dim 768 \
#     --use_enhanced_semgcn \
#     --use_contrastive \
#     --contrastive_weight 0.1

### 10b. Restaurant 2015 Dataset

In [ ]:
# Uncomment to test on Restaurant 2015 dataset
# !python train.py \
#     --seed 42 \
#     --max_span_size 8 \
#     --batch_size 16 \
#     --epochs 120 \
#     --dataset 15res \
#     --pretrained_deberta_name microsoft/deberta-v3-base \
#     --deberta_feature_dim 768 \
#     --hidden_dim 384 \
#     --emb_dim 768 \
#     --use_enhanced_semgcn \
#     --use_contrastive \
#     --contrastive_weight 0.1

## 11. View Training Logs

In [ ]:
!ls -lh log/14res/
!echo "\n=== Latest Training Log ==="
!tail -100 log/14res/train_*.log

## 12. Check Best Model Results

In [ ]:
import json
import glob

# Find latest results file
result_files = glob.glob('log/14res/*.json')
if result_files:
    latest_file = max(result_files, key=lambda x: os.path.getmtime(x))
    print(f"Latest results: {latest_file}\n")
    
    with open(latest_file, 'r') as f:
        results = json.load(f)
    
    print("=== Best Results ===")
    print(f"Entity F1: {results.get('entity_f1', 'N/A')}")
    print(f"Triplet F1: {results.get('triplet_f1', 'N/A')}")
    print(f"Best Epoch: {results.get('best_epoch', 'N/A')}")
else:
    print("No results found yet. Run training first.")

## 13. Performance Summary

### Expected Results:

| Configuration | Entity F1 | Triplet F1 | Improvement |
|--------------|-----------|------------|-------------|
| Baseline (Original) | 87.65% | 75.75% | --- |
| + Enhanced SemGCN | 88.68% | 77.14% | +1.39% |
| + SemGCN + Contrastive | **88.5-89.0%** | **77.6-78.0%** | **+1.85-2.25%** |

### Key Features:
- ✅ Enhanced Semantic GCN with relative position encoding
- ✅ Contrastive learning for entity-opinion pairing
- ✅ InfoNCE loss for better representation learning
- ✅ Configurable contrastive weight (default: 0.1)

### Hyperparameters:
- `--use_contrastive`: Enable contrastive learning
- `--contrastive_weight`: Weight for contrastive loss (0.05-0.2)
- Temperature: 0.07 (hardcoded in SimplifiedContrastiveLoss)

### Next Steps:
1. Run full training with contrastive learning
2. Compare with baseline results
3. Tune hyperparameters if needed
4. Test on other datasets (14lap, 15res, 16res)